---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    ans = []
    for i in P1_Graphs:
        degrees = i.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(x)/float(nx.number_of_nodes(i)) for x in degree_values]
        corr_coef = np.corrcoef(np.log(degree_values), np.log(histogram))
        if abs(corr_coef[0][1]) >= 0.85: ### using the corrcoef to find the linear relationship between the logscale of degree and degree distribution, set threshold at 0.85, if above, then this is PA, else, check for small world
            ans.append("PA")
            continue
        avg_path = nx.average_shortest_path_length(i)
        avg_clustering = nx.average_clustering(i)
        if avg_path < 10 and avg_clustering > 0.3: ### Small world with low avg_path and high avg_clustering has low rewiring prob
            ans.append("SW_L")
        else:
            ans.append("SW_H")
        
    return ans# Your Answer Here

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    df = pd.DataFrame(index=G.nodes())

    # df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

    ### get node based features
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['close_Cent'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['Btwn_Cent'] = pd.Series(nx.betweenness_centrality(G, normalized=True,endpoints=False))


    df_test = df[pd.isna(df['ManagementSalary'])]
    df_train = df[df['ManagementSalary'].notna()]

    X_train = df_train.drop(['ManagementSalary'], axis = 1)
    y_train = df_train['ManagementSalary']

    X_test = df_test.drop(['ManagementSalary'], axis = 1)
    y_test = df_test['ManagementSalary']

    LG = linear_model.LogisticRegression()
    grid = {'C': np.power(10.0, np.arange(-10, 10, 2))}
    clf = GridSearchCV(LG, grid, scoring = 'roc_auc', cv=5)

    clf.fit(X_train.values, y_train.values)

    print("best_train_score:{}".format(clf.best_score_))

    print("best_estimator:{}".format(clf.best_estimator_))

    return pd.Series(clf.best_estimator_.predict_proba(X_test.values)[:,1], index=df_test.index)
    

In [6]:
salary_predictions()

best_train_score:0.9508932339849342
best_estimator:LogisticRegression(C=10000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


1       4.252606e-02
2       6.789370e-01
5       9.994968e-01
8       1.180754e-01
14      1.255550e-01
18      2.869811e-01
27      2.243543e-01
30      3.746245e-01
31      9.462036e-02
34      7.982393e-02
37      2.006442e-02
40      5.395771e-02
45      1.134710e-01
54      2.364442e-01
55      5.078089e-01
60      3.645217e-02
62      9.979904e-01
65      7.972525e-01
77      5.564268e-02
79      1.368315e-01
97      4.699410e-02
101     1.353065e-02
103     2.269344e-01
108     8.983789e-02
113     4.613058e-01
122     3.248237e-03
141     3.331975e-01
142     8.381176e-01
144     5.457566e-02
145     6.820005e-01
            ...     
913     4.332014e-03
914     2.309755e-02
915     5.000997e-07
918     1.111416e-01
923     7.728191e-04
926     4.125175e-02
931     1.196856e-02
934     3.572705e-04
939     1.101816e-04
944     1.011354e-06
945     2.445758e-03
947     1.279206e-01
950     7.862953e-02
951     1.683887e-03
953     1.737122e-03
959     3.676942e-06
962     8.353

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [10]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [13]:
future_connections['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
future_connections['common neighbors'] = [len(list(nx.common_neighbors(G, i[0], i[1]))) for i in future_connections.index]
future_connections['Communit common neighbors score'] = [list(nx.cn_soundarajan_hopcroft(G, [i], community='Department'))[0][2] for i in future_connections.index]
future_connections['shortest path length'] = [nx.shortest_path_length(G, i[0], i[1]) if nx.has_path(G, i[0], i[1]) else 0 for i in future_connections.index]

In [16]:
def new_connections_predictions():
    df_test = future_connections[pd.isna(future_connections['Future Connection'])]
    df_train = future_connections[future_connections['Future Connection'].notna()]

    X_train = df_train.drop(['Future Connection'], axis = 1)
    y_train = df_train['Future Connection']

    X_test = df_test.drop(['Future Connection'], axis = 1)

    LG = linear_model.LogisticRegression()
    LG.fit(X_train, y_train)
    print("score:{}".format(roc_auc_score(y_train.values, LG.predict_proba(X_train.values)[:,1])))
    return pd.Series(LG.predict_proba(X_test)[:,1], index = df_test.index)
# grid = {'C': np.power(10.0, np.arange(-0, 2))}
# clf = GridSearchCV(LG, grid, scoring = 'roc_auc', cv=2)

# clf.fit(X_train, y_train)

# print("best_train_score:{}".format(clf.best_score_))

# print("best_estimator:{}".format(clf.best_estimator_))

# pd.Series(clf.best_estimator_.predict_proba(X_test)[:,1], index=df_test.index)

In [17]:
new_connections_predictions()

score:0.9122492476199151


(107, 348)    0.029469
(542, 751)    0.012122
(20, 426)     0.597083
(50, 989)     0.012230
(942, 986)    0.013446
(324, 857)    0.012215
(13, 710)     0.154904
(19, 271)     0.127766
(319, 878)    0.012268
(659, 707)    0.012133
(49, 843)     0.012302
(208, 893)    0.012159
(377, 469)    0.008226
(405, 999)    0.019239
(129, 740)    0.017737
(292, 618)    0.017972
(239, 689)    0.013368
(359, 373)    0.009714
(53, 523)     0.291610
(276, 984)    0.012297
(202, 997)    0.013415
(604, 619)    0.033348
(270, 911)    0.012274
(261, 481)    0.068842
(200, 450)    0.999966
(213, 634)    0.013164
(644, 735)    0.033073
(346, 553)    0.011820
(521, 738)    0.011311
(422, 953)    0.017564
                ...   
(672, 848)    0.012274
(28, 127)     0.945197
(202, 661)    0.011774
(54, 195)     0.999667
(295, 864)    0.012199
(814, 936)    0.012105
(839, 874)    0.012348
(139, 843)    0.012208
(461, 544)    0.010961
(68, 487)     0.010901
(622, 932)    0.012172
(504, 936)    0.016350
(479, 528) 